In [1]:
import pandas as pd

def fix_column_names(label_path, save_path=None):
    """
    Fix column names in the label file by standardizing them.
    
    Args:
        label_path (str): Path to the original label CSV file
        save_path (str, optional): Path to save the cleaned CSV file. 
                                 If None, will return the DataFrame
    
    Returns:
        pd.DataFrame if save_path is None, else None
    """
    # Read the label file
    df = pd.read_csv(label_path)
    
    # Print original columns
    print("Original columns:", df.columns.tolist())
    
    # Clean column names:
    # 1. Convert to lowercase
    # 2. Remove extra spaces
    # 3. Replace spaces with underscores
    df.columns = df.columns.str.lower().str.strip().str.replace(' ', '_')
    
    # Map variations to standard names
    column_mapping = {
        'songid': 'song_id',
        'song_id': 'song_id',
        'valencemean': 'valence_mean',
        'valence_mean': 'valence_mean',
        'valencestd': 'valence_std',
        'valence_std': 'valence_std',
        'arousalmean': 'arousal_mean',
        'arousal_mean': 'arousal_mean',
        'arousalstd': 'arousal_std',
        'arousal_std': 'arousal_std'
    }
    
    # Rename columns based on mapping
    #df = df.rename(columns=column_mapping)
    df.columns = df.columns.str.strip()

    
    # Print new columns
    print("Cleaned columns:", df.columns.tolist())
    
    # Check if all required columns are present
    required_columns = ['song_id', 'valence_mean', 'valence_std', 'arousal_mean', 'arousal_std']
    missing_columns = [col for col in required_columns if col not in df.columns]
    
    if missing_columns:
        raise ValueError(f"Missing required columns: {missing_columns}")
    
    if save_path:
        df.to_csv(save_path, index=False)
        print(f"Saved cleaned data to {save_path}")
    else:
        return df

# Example usage
if __name__ == "__main__":
    # Fix training labels
    train_label_path = "/kaggle/input/deam-mediaeval-dataset-emotional-analysis-in-music/DEAM_Annotations/annotations/annotations averaged per song/song_level/static_annotations_averaged_songs_1_2000.csv"
    test_label_path = "/kaggle/input/deam-mediaeval-dataset-emotional-analysis-in-music/DEAM_Annotations/annotations/annotations averaged per song/song_level/static_annotations_averaged_songs_2000_2058.csv"
    
    # Fix and save training labels
    fix_column_names(
        train_label_path, 
        save_path="clean_train_labels.csv"
    )
    
    # Fix and save test labels
    fix_column_names(
        test_label_path, 
        save_path="clean_test_labels.csv"
    )

Original columns: ['song_id', ' valence_mean', ' valence_std', ' arousal_mean', ' arousal_std']
Cleaned columns: ['song_id', 'valence_mean', 'valence_std', 'arousal_mean', 'arousal_std']
Saved cleaned data to clean_train_labels.csv
Original columns: ['song_id', ' valence_mean', ' valence_std', ' valence_ max_mean', ' valence_max_std', ' valence_min_mean', ' valence_min_std', ' arousal_mean', ' arousal_std', ' arousal_max_mean', ' arousal_max_std', ' arousal_min_mean', ' arousal_min_std']
Cleaned columns: ['song_id', 'valence_mean', 'valence_std', 'valence__max_mean', 'valence_max_std', 'valence_min_mean', 'valence_min_std', 'arousal_mean', 'arousal_std', 'arousal_max_mean', 'arousal_max_std', 'arousal_min_mean', 'arousal_min_std']
Saved cleaned data to clean_test_labels.csv


In [2]:
import torch
from torch import nn
import pandas as pd
import numpy as np
from torch.utils.data import Dataset, DataLoader
import os
from sklearn.preprocessing import StandardScaler

class AudioEmotionDataset(Dataset):
    def __init__(self, feature_dir, label_path, scaler=None, train=True):
        """
        Args:
            feature_dir (str): Single directory containing all feature CSV files
            label_path (str): Path to label CSV file (train or test)
            scaler (StandardScaler, optional): Scaler for feature normalization
            train (bool): If True, fit_transform scaler; if False, transform only
        """
        self.feature_dir = feature_dir
        self.label_df = pd.read_csv(label_path)
        self.song_ids = self.label_df['song_id'].values
        self.train = train
        
        # Load and process features for songs that have labels
        self.features = []
        self.valid_indices = []  # Keep track of songs that have both features and labels
        self.valid_song_ids = []
        
        for idx, song_id in enumerate(self.song_ids):
            feature_path = os.path.join(feature_dir, f"{song_id}_features.csv")
            if os.path.exists(feature_path):
                try:
                    song_features = pd.read_csv(feature_path)
                    # Remove end_time column
                    song_features = song_features.drop('end_time', axis=1)
                    # Calculate mean of features across all segments
                    mean_features = song_features.mean().values
                    self.features.append(mean_features)
                    self.valid_indices.append(idx)
                    self.valid_song_ids.append(song_id)
                except Exception as e:
                    print(f"Error processing {song_id}: {e}")
                    continue
        
        self.features = np.array(self.features)
        
        # Keep only labels for songs that have features
        self.label_df = self.label_df.iloc[self.valid_indices]
        
        # Scale features
        if scaler is None and train:
            self.scaler = StandardScaler()
            self.features = self.scaler.fit_transform(self.features)
        elif scaler is not None and not train:
            self.scaler = scaler
            self.features = self.scaler.transform(self.features)
        
        # Prepare labels
        self.labels = self.label_df[['valence_mean', 'valence_std', 
                                   'arousal_mean', 'arousal_std']].values
        
        print(f"Loaded {len(self.features)} songs with both features and labels")
    
    def __len__(self):
        return len(self.features)
    
    def __getitem__(self, idx):
        features = torch.FloatTensor(self.features[idx])
        labels = torch.FloatTensor(self.labels[idx])
        return features, labels

class EmotionPredictor(nn.Module):
    def __init__(self, input_size):
        super(EmotionPredictor, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(input_size, 64),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(64, 32),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(32, 4)  # 4 outputs: valence_mean, valence_std, arousal_mean, arousal_std
        )
    
    def forward(self, x):
        return self.model(x)

def train_model(train_loader, val_loader, model, num_epochs=100, learning_rate=0.001):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"Using device: {device}")
    
    model = model.to(device)
    criterion = nn.MSELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
    
    best_val_loss = float('inf')
    best_model = None
    
    for epoch in range(num_epochs):
        # Training phase
        model.train()
        train_loss = 0
        for features, labels in train_loader:
            features, labels = features.to(device), labels.to(device)
            
            optimizer.zero_grad()
            outputs = model(features)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            
            train_loss += loss.item()
        
        # Validation phase
        model.eval()
        val_loss = 0
        with torch.no_grad():
            for features, labels in val_loader:
                features, labels = features.to(device), labels.to(device)
                outputs = model(features)
                val_loss += criterion(outputs, labels).item()
        
        train_loss /= len(train_loader)
        val_loss /= len(val_loader)
        
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            best_model = model.state_dict().copy()
        
        if (epoch + 1) % 10 == 0:
            print(f'Epoch [{epoch+1}/{num_epochs}], Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}')
    
    return best_model

# Example usage
def main():
    # Set paths
    feature_dir = "/kaggle/input/deam-featuresmfcc-rms"  # Single directory containing all features
    train_labels_path = "/kaggle/working/clean_train_labels.csv"
    test_labels_path = "/kaggle/working/clean_train_labels.csv"
    
    # Create datasets
    train_dataset = AudioEmotionDataset(feature_dir, train_labels_path, train=True)
    test_dataset = AudioEmotionDataset(feature_dir, test_labels_path, 
                                     scaler=train_dataset.scaler, train=False)
    
    # Create data loaders
    train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)
    
    # Initialize model
    input_size = train_dataset.features.shape[1]  # Number of features
    model = EmotionPredictor(input_size)
    
    # Train model
    best_model_state = train_model(train_loader, test_loader, model)
    
    # Save best model
    torch.save(best_model_state, 'best_emotion_model.pth')
    
    # Save scaler for future use
    import joblib
    joblib.dump(train_dataset.scaler, 'feature_scaler.pkl')

if __name__ == "__main__":
    main()

Loaded 1744 songs with both features and labels
Loaded 1744 songs with both features and labels
Using device: cuda
Epoch [10/100], Train Loss: 0.8296, Val Loss: 0.4915
Epoch [20/100], Train Loss: 0.7114, Val Loss: 0.4791
Epoch [30/100], Train Loss: 0.7103, Val Loss: 0.4737
Epoch [40/100], Train Loss: 0.6568, Val Loss: 0.4904
Epoch [50/100], Train Loss: 0.6428, Val Loss: 0.4657
Epoch [60/100], Train Loss: 0.6221, Val Loss: 0.4689
Epoch [70/100], Train Loss: 0.6175, Val Loss: 0.4637
Epoch [80/100], Train Loss: 0.5812, Val Loss: 0.4563
Epoch [90/100], Train Loss: 0.5870, Val Loss: 0.4722
Epoch [100/100], Train Loss: 0.5623, Val Loss: 0.4568
